In [1]:
import pandas as pd
import numpy as np

In [2]:
def filter_data(dimension, step, cpu, gpu, mem, cores, core_vals, cpu_vals, gpu_vals, mem_vals):


    if step == 'next':
        if dimension == 'cores':
            # Get index of cores from core_vals
            core_index = core_vals.index(cores)
            last_index = len(core_vals) - 1
            new_index = int(np.ceil((core_index+last_index)/2))
            value = core_vals[new_index]
            core_vals = core_vals[core_index+1:]

        elif dimension == 'cpu':
            # Get index of cpu from cpu_vals
            cpu_index = cpu_vals.index(cpu)
            last_index = len(cpu_vals) - 1
            new_index = int(np.ceil((cpu_index+last_index)/2))
            value = cpu_vals[new_index]
            cpu_vals = cpu_vals[cpu_index+1:]

        elif dimension == 'gpu':
            # Get index of gpu from gpu_vals
            gpu_index = gpu_vals.index(gpu)
            last_index = len(gpu_vals) - 1
            new_index = int(np.ceil((gpu_index+last_index)/2))
            value = gpu_vals[new_index]
            gpu_vals = gpu_vals[gpu_index+1:]

        else:
            # Get index of mem from mem_vals
            mem_index = mem_vals.index(mem)
            last_index = len(mem_vals) - 1
            new_index = int(np.ceil((mem_index+last_index)/2))
            value = mem_vals[new_index]
            mem_vals = mem_vals[mem_index+1:]

    else:

        if dimension == 'cores':
            # Get index of cores from core_vals
            core_index = core_vals.index(cores)
            new_index = int(np.floor(core_index/2))
            value = core_vals[new_index]
            core_vals = core_vals[:core_index]
        
        elif dimension == 'cpu':
            # Get index of cpu from cpu_vals
            cpu_index = cpu_vals.index(cpu)
            new_index = int(np.floor(cpu_index/2))
            value = cpu_vals[new_index]
            cpu_vals = cpu_vals[:cpu_index]

        elif dimension == 'gpu':
            # Get index of gpu from gpu_vals
            gpu_index = gpu_vals.index(gpu)
            new_index = int(np.floor(gpu_index/2))
            value = gpu_vals[new_index]
            gpu_vals = gpu_vals[:gpu_index]

        else:
            # Get index of mem from mem_vals
            mem_index = mem_vals.index(mem)
            new_index = int(np.floor(mem_index/2))
            value = mem_vals[new_index]
            mem_vals = mem_vals[:mem_index]

            # mem_index = mem_vals.index(mem)
            # new_index = int(np.floor(len(mem_vals)/2))
            # value = mem_index
            # mem_vals = mem_vals[:mem_index+1]

    return value, core_vals, cpu_vals, gpu_vals, mem_vals


In [3]:
def optimize(data, power_budget, time_budget, runs):

    core_vals=[4, 8, 12] #3 possible values
    gpu_vals=[114750000, 318750000, 522750000, 726750000, 930750000, 1134750000, 1300500000]
    cpu_vals=[422400, 729600, 1036800, 1344000, 1651200, 1958400, 2201600] #in kHz, 7 possible values
    mem_vals = [665600000, 2133000000, 3199000000]

    best_powermode = str(8) + '_' + str(1344000) + '_' + str(726750000) + '_' + str(2133000000)
    mid_powermode = best_powermode
    best_power = 0
    best_time = 99999

    core_mid = 8
    cpu_mid = 1344000
    gpu_mid = 726750000
    mem_mid = 2133000000 

    curr_core = core_mid
    curr_cpu = cpu_midx 
    curr_gpu = gpu_mid
    curr_mem = mem_mid

    mid_index = data[(data['cores'] == core_mid) & (data['cpu'] == cpu_mid) & (data['gpu'] == gpu_mid) & (data['mem'] == mem_mid)].index[0]
    curr_index = mid_index
    curr_index_power = data.loc[mid_index, 'observed_power']
    curr_index_time = data.loc[mid_index, 'observed_time']

    if best_time > curr_index_time and curr_index_power <= power_budget:
        best_power = curr_index_power
        best_time = curr_index_time
        best_powermode = mid_powermode

    order = ['cpu', 'gpu', 'cores', 'mem']
       # curr_order = [curr_gpu, curr_cpu, curr_core, curr_mem]
    start_powermode = str(curr_core) + '_' + str(curr_cpu) + '_' + str(curr_gpu) + '_' + str(curr_mem)
    print("Start powermode: ",start_powermode)
    space = len(core_vals) * len(cpu_vals) * len(gpu_vals) * len(mem_vals)
    print("Space left: ", space)
    for i in range(runs):
        count = 0
        for i in order:
            
            if curr_index_power <= power_budget:

                if i == 'gpu':
                    curr_gpu, core_vals, cpu_vals, gpu_vals, mem_vals  = filter_data(i,'next', curr_cpu, curr_gpu, curr_mem, curr_core, core_vals, cpu_vals, gpu_vals, mem_vals)
                elif i == 'cpu':
                    curr_cpu, core_vals, cpu_vals, gpu_vals, mem_vals = filter_data(i,'next', curr_cpu, curr_gpu, curr_mem, curr_core, core_vals, cpu_vals, gpu_vals, mem_vals)
                elif i == 'mem':
                    curr_mem, core_vals, cpu_vals, gpu_vals, mem_vals = filter_data(i, 'next',curr_cpu, curr_gpu, curr_mem, curr_core, core_vals, cpu_vals, gpu_vals, mem_vals)
                else:
                    curr_core, core_vals, cpu_vals, gpu_vals, mem_vals = filter_data(i,'next', curr_cpu, curr_gpu, curr_mem, curr_core, core_vals, cpu_vals, gpu_vals, mem_vals)

                curr_index_power = data[(data['cores'] == curr_core) & (data['cpu'] == curr_cpu) & (data['gpu'] == curr_gpu) & (data['mem'] == curr_mem)]['observed_power'].values[0]
                curr_index_time = data[(data['cores'] == curr_core) & (data['cpu'] == curr_cpu) & (data['gpu'] == curr_gpu) & (data['mem'] == curr_mem)]['observed_time'].values[0]
                powermode = str(curr_core) + '_' + str(curr_cpu) + '_' + str(curr_gpu) + '_' + str(curr_mem)
                print("Current Powermode: ", powermode)
                print("Current Power: ", curr_index_power)
                print("Current Time: ", curr_index_time)

                print("Current Core Vals: ", core_vals)
                print("Current CPU Vals: ", cpu_vals)
                print("Current GPU Vals: ", gpu_vals)
                print("Current Mem Vals: ", mem_vals)

                space = len(core_vals) * len(cpu_vals) * len(gpu_vals) * len(mem_vals)
                print("Space left: ", space)

                if best_time > curr_index_time and curr_index_power <= power_budget:
                    best_power = curr_index_power
                    best_time = curr_index_time
                    best_powermode = powermode
                
                if space == 0:
                    break

            else:

                if i == 'gpu':
                    curr_gpu, core_vals, cpu_vals, gpu_vals, mem_vals = filter_data(i, 'prev',curr_cpu, curr_gpu, curr_mem, curr_core, core_vals, cpu_vals, gpu_vals, mem_vals)
                elif i == 'cpu':
                    curr_cpu, core_vals, cpu_vals, gpu_vals, mem_vals = filter_data(i, 'prev',curr_cpu, curr_gpu, curr_mem, curr_core, core_vals, cpu_vals, gpu_vals, mem_vals)
                elif i == 'mem':
                    curr_mem, core_vals, cpu_vals, gpu_vals, mem_vals = filter_data(i, 'prev',curr_cpu, curr_gpu, curr_mem, curr_core, core_vals, cpu_vals, gpu_vals, mem_vals)
                else:
                    curr_core, core_vals, cpu_vals, gpu_vals, mem_vals = filter_data(i, 'prev',curr_cpu, curr_gpu, curr_mem, curr_core, core_vals, cpu_vals, gpu_vals, mem_vals)

                curr_index_power = data[(data['cores'] == curr_core) & (data['cpu'] == curr_cpu) & (data['gpu'] == curr_gpu) & (data['mem'] == curr_mem)]['observed_power'].values[0]    
                curr_index_time = data[(data['cores'] == curr_core) & (data['cpu'] == curr_cpu) & (data['gpu'] == curr_gpu) & (data['mem'] == curr_mem)]['observed_time'].values[0]
                powermode = str(curr_core) + '_' + str(curr_cpu) + '_' + str(curr_gpu) + '_' + str(curr_mem)
                print("Current Powermode: ", powermode)
                print("Current Power: ", curr_index_power)
                print("Current Time: ", curr_index_time)

                print("Current Core Vals: ", core_vals)
                print("Current CPU Vals: ", cpu_vals)
                print("Current GPU Vals: ", gpu_vals)
                print("Current Mem Vals: ", mem_vals)

                space = len(core_vals) * len(cpu_vals) * len(gpu_vals) * len(mem_vals)
                print("Space left: ", space)

                if best_time > curr_index_time and curr_index_power <= power_budget:
                    best_power = curr_index_power
                    best_time = curr_index_time
                    best_powermode = powermode   

                if space == 0:
                    break  
                    

            count+=1

    print("Best Powermode: ", best_powermode)
    print("Best Power: ", best_power)
    print("Best Time: ", best_time) 
    return best_powermode 
        

In [4]:
model = 'resnet'
data = pd.read_csv(f'{model}_train_data_final.csv')

In [5]:
core_vals=[4, 8, 12] #3 possible values
gpu_vals=[114750000, 318750000, 522750000, 726750000, 930750000, 1134750000, 1300500000]
cpu_vals=[422400, 729600, 1036800, 1344000, 1651200, 1958400, 2201600] #in kHz, 7 possible values
mem_vals = [665600000, 2133000000, 3199000000]

# Filter data based on values in the list only
data = data[data['cores'].isin(core_vals)]
data = data[data['gpu'].isin(gpu_vals)]
data = data[data['cpu'].isin(cpu_vals)]
data = data[data['mem'].isin(mem_vals)]

In [6]:
data

,cores,cpu,gpu,mem,observed_time,observed_power
781,4,422400,1300500000,2133000000,216.409889,22.8145
782,4,422400,1300500000,3199000000,257.015106,22.5720
783,4,422400,1300500000,665600000,299.540268,22.1700
789,4,422400,1134750000,2133000000,254.655281,20.8105
790,4,422400,1134750000,3199000000,233.582397,20.4630
...,...,...,...,...,...,...
4358,12,2201600,318750000,3199000000,171.996704,23.8930
4359,12,2201600,318750000,665600000,236.020462,20.2730
4365,12,2201600,114750000,2133000000,465.527954,17.7560
4366,12,2201600,114750000,3199000000,464.036285,18.5690


In [7]:
optimize(data, 30, 100000, 2)

Start powermode:  8_1344000_726750000_2133000000
Space left:  441
Current Powermode:  8_1958400_726750000_2133000000
Current Power:  28.7515
Current Time:  94.62198257446288
Current Core Vals:  [4, 8, 12]
Current CPU Vals:  [1651200, 1958400, 2201600]
Current GPU Vals:  [114750000, 318750000, 522750000, 726750000, 930750000, 1134750000, 1300500000]
Current Mem Vals:  [665600000, 2133000000, 3199000000]
Space left:  189
Current Powermode:  8_1958400_1134750000_2133000000
Current Power:  36.909
Current Time:  79.08363342285156
Current Core Vals:  [4, 8, 12]
Current CPU Vals:  [1651200, 1958400, 2201600]
Current GPU Vals:  [930750000, 1134750000, 1300500000]
Current Mem Vals:  [665600000, 2133000000, 3199000000]
Space left:  81
Current Powermode:  4_1958400_1134750000_2133000000
Current Power:  36.393
Current Time:  81.22697830200195
Current Core Vals:  [4]
Current CPU Vals:  [1651200, 1958400, 2201600]
Current GPU Vals:  [930750000, 1134750000, 1300500000]
Current Mem Vals:  [665600000, 

'8_1958400_726750000_2133000000'

In [26]:
# Power list rom 10 to 50 in increments of 1
if model =='bert':
    power_list = list(range(10, 61))
else:
    power_list = list(range(10, 51))
time_budget = 100000

In [27]:
df_1 = pd.DataFrame(columns=['cpu','gpu','mem','cores','observed_time','observed_power','power_budget','num_pwds'])

for i in power_list:
    powermode = optimize(data, i, time_budget, 2)   
    powermode = powermode.split('_')
    cpu = int(powermode[1])
    gpu = int(powermode[2])
    mem = int(powermode[3])
    cores = int(powermode[0])
    res_df = data[(data['cores'] == cores) & (data['cpu'] == cpu) & (data['gpu'] == gpu) & (data['mem'] == mem)]

    time = res_df['observed_time'].values[0]
    power = res_df['observed_power'].values[0]

    if power > i:
        low_pwmd = data[(data['cores'] == 4) & (data['gpu'] == 114750000) & (data['cpu'] == 422400) & (data['mem'] == 665600000)]
        if low_pwmd['observed_power'].values[0] < i:
            df_1.loc[len(df_1)] = [4, 422400, 114750000, 665600000, low_pwmd['observed_time'].values[0], low_pwmd['observed_power'].values[0], i, 7]
        else:
            df_1.loc[len(df_1)] = [0, 0, 0, 0, 0, 0, i, 7]
        # high_slope_df.loc[len(high_slope_df)] = [0, power_budget]
    else:
        df_1.loc[len(df_1)] = [cpu, gpu, mem, cores, time, power, i, 7]

Start powermode:  8_1344000_726750000_2133000000
Space left:  441
Current Powermode:  8_729600_726750000_2133000000
Current Power:  22.271
Current Time:  196.4656982421875
Current Core Vals:  [4, 8, 12]
Current CPU Vals:  [422400, 729600, 1036800]
Current GPU Vals:  [114750000, 318750000, 522750000, 726750000, 930750000, 1134750000, 1300500000]
Current Mem Vals:  [665600000, 2133000000, 3199000000]
Space left:  189
Current Powermode:  8_729600_318750000_2133000000
Current Power:  18.776
Current Time:  304.2663269042969
Current Core Vals:  [4, 8, 12]
Current CPU Vals:  [422400, 729600, 1036800]
Current GPU Vals:  [114750000, 318750000, 522750000]
Current Mem Vals:  [665600000, 2133000000, 3199000000]
Space left:  81
Current Powermode:  4_729600_318750000_2133000000
Current Power:  18.675
Current Time:  305.9073486328125
Current Core Vals:  [4]
Current CPU Vals:  [422400, 729600, 1036800]
Current GPU Vals:  [114750000, 318750000, 522750000]
Current Mem Vals:  [665600000, 2133000000, 3199

In [28]:
def find_optimal_pm(df, tmax, pmax):
    #filter rows where interleaved_power<=pmax, sort in ascending order of interleaved_time
    df = df[df['observed_power'] <= pmax]
    df = df.sort_values(by='observed_time', ascending=True)
    #print top 10 rows with all columns
    # pd.set_option('display.max_columns', 50)
    # print(df.head(20))
    #return top1 powermode, time and power
    return df.iloc[0]['cores'], df.iloc[0]['cpu'], df.iloc[0]['gpu'], df.iloc[0]['mem'], df.iloc[0]['observed_time'], df.iloc[0]['observed_power']

In [29]:
data.reset_index(drop=True, inplace=True)

In [30]:
df_2 = pd.DataFrame(columns=['cpu','gpu','mem','cores','observed_time','observed_power','power_budget'])

for i in power_list:
    try:
        cores, cpu, gpu, mem, time, power = find_optimal_pm(data, time_budget, i) 
        cores = int(cores)
        cpu = int(cpu)
        gpu = int(gpu)
        mem = int(mem)
        res_df = data[(data['cores'] == cores) & (data['cpu'] == cpu) & (data['gpu'] == gpu) & (data['mem'] == mem)]

        time = res_df['observed_time'].values[0]
        power = res_df['observed_power'].values[0]
        df_2.loc[len(df_2)] = [cpu, gpu, mem, cores, time, power, i]
        # df_2.loc[len(df_2)] = [0, 0, 0, 0, 0, 0, i]
    except:
        df_2.loc[len(df_2)] = [0, 0, 0, 0, 0, 0, i]

In [31]:
merged_df = pd.merge(df_1, df_2, on='power_budget', suffixes=('_bs', '_optim'))

In [32]:
merged_df.to_csv(f'{model}_train_only_bs_vs_optim.csv', index=False)
merged_df

,cpu_bs,gpu_bs,mem_bs,cores_bs,observed_time_bs,observed_power_bs,power_budget,num_pwds,cpu_optim,gpu_optim,mem_optim,cores_optim,observed_time_optim,observed_power_optim
0,0.0,0.000000e+00,0.000000e+00,0.0,0.000000,0.000,10.0,7.0,0.0,0.000000e+00,0.000000e+00,0.0,0.000000,0.0000
1,0.0,0.000000e+00,0.000000e+00,0.0,0.000000,0.000,11.0,7.0,0.0,0.000000e+00,0.000000e+00,0.0,0.000000,0.0000
2,0.0,0.000000e+00,0.000000e+00,0.0,0.000000,0.000,12.0,7.0,0.0,0.000000e+00,0.000000e+00,0.0,0.000000,0.0000
3,0.0,0.000000e+00,0.000000e+00,0.0,0.000000,0.000,13.0,7.0,0.0,0.000000e+00,0.000000e+00,0.0,0.000000,0.0000
4,0.0,0.000000e+00,0.000000e+00,0.0,0.000000,0.000,14.0,7.0,0.0,0.000000e+00,0.000000e+00,0.0,0.000000,0.0000
5,422400.0,1.147500e+08,6.656000e+08,4.0,806.754791,14.149,15.0,7.0,1651200.0,1.147500e+08,6.656000e+08,4.0,785.293915,14.8510
6,422400.0,3.187500e+08,6.656000e+08,4.0,465.128036,15.956,16.0,7.0,422400.0,3.187500e+08,6.656000e+08,4.0,465.128036,15.9560
7,1036800.0,3.187500e+08,6.656000e+08,4.0,381.768250,16.958,17.0,7.0,1036800.0,3.187500e+08,6.656000e+08,4.0,381.768250,16.9580
8,1036800.0,5.227500e+08,6.656000e+08,4.0,328.194077,17.670,18.0,7.0,1036800.0,5.227500e+08,6.656000e+08,4.0,328.194077,17.6700
9,729600.0,3.187500e+08,2.133000e+09,8.0,304.266327,18.776,19.0,7.0,1344000.0,3.187500e+08,2.133000e+09,4.0,297.827988,18.8730
